# Resume RAG Agent with SMS Notification
Welcome to the Resume RAG Agent Lab!

## Introduction
In this lab, you'll build an autonomous Resume RAG (Retrieval-Augmented Generation) chatbot agent. The agent answers interview questions using information from a resume PDF. If the answer isn't found, it sends an SMS notification to the user. You'll learn how to:
- Load and process a resume PDF
- Use semantic search with embeddings
- Integrate Twilio for SMS notifications
- Run an async chat loop

---
<div style="border: 2px solid #ccc; border-radius: 12px; padding: 16px; background: #111; display: flex; align-items: center; gap: 24px;">
    <img src="images/robo7.png" alt="Robo7" style="width:120px;"/>
    <div>
        <h3 style="color:#fff;">Meet Robo7!</h3>
        <p style="color:#eee;">If the Resume RAG Agent can't find an answer in your resume, Robo7 will send you an SMS to let you know! 🤖📱</p>
        <em style="color:#bbb;">"When in doubt, Robo7 texts it out!"</em>
    </div>
</div>

## Resume RAG Agent Implementation

This section implements the Resume RAG Agent using LangChain, OpenAI, FAISS, and Twilio. The agent answers interview questions using your resume PDF. If the answer is not found, it sends an SMS notification using Twilio.

**Workflow:**
- Load resume PDF and split into chunks
- Embed and index chunks for semantic search
- Define tools for resume search and SMS notification
- Create a conversational agent with a custom prompt
- Run an interactive chat loop

In [1]:
from twilio.rest import Client
from langchain.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
import logging
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment
openai_api_key = os.getenv("OPENAI_API_KEY")

# Twilio credentials
twilio_sid = os.getenv("TWILIO_ACCOUNT_SID")
twilio_token = os.getenv("TWILIO_AUTH_TOKEN")
twilio_from = os.getenv("TWILIO_FROM_NUMBER")
twilio_to = os.getenv("TWILIO_TO_NUMBER")

# Create twilio client
twilio_client = Client(twilio_sid, twilio_token)

def send_sms_tool(message: str) -> str:
    print(f"[DEBUG] Sending SMS: {message}")
    twilio_client.messages.create(
        body=message,
        from_=twilio_from,
        to=twilio_to
    )
    return "SMS sent to your phone."

twilio_tool = Tool(
    name="twilio_tool",
    func=send_sms_tool,
    description="If you ever answer 'I don't know based on my resume.', immediately use this tool to notify the user by SMS. When you use this tool, send the original interview question that was asked, not the fallback answer."
)

def resume_search_tool(query: str) -> str:
    result = qa_chain.invoke({"query": query})
    if isinstance(result, dict):
        answer = str(result)
    print(f"[DEBUG] Resume search result: {answer}")
    if answer.strip() == "" or "I don't know" in answer:
        return "I don't know based on my resume."
    return answer

resume_tool = Tool(
    name="resume_search",
    func=resume_search_tool,
    description="Use this tool to search the resume and answer interview questions based on its content. If the answer is not found, return 'I don't know based on my resume.' as a string. If you ever answer 'I don't know based on my resume.', you must immediately use the twilio_tool to notify the user by SMS."
)

pdf_path = "my_resume.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

llm = ChatOpenAI(model="gpt-4", temperature=0, api_key=openai_api_key)

prompt_template = """
You are answering interview questions based on the resume.
If the answer is not in the resume, say "I don't know based on my resume." If you ever answer 'I don't know based on my resume.', you must immediately use the twilio_tool to notify the user by SMS. When you use the twilio_tool, send the original interview question that was asked, not the fallback answer.
Question: {input}
{agent_scratchpad}
Answer as if you are the candidate:
"""

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False
)

agent = create_tool_calling_agent(llm, [resume_tool, twilio_tool], PromptTemplate.from_template(prompt_template))
executor = AgentExecutor(agent=agent, tools=[resume_tool, twilio_tool])

## Async Chat Loop

The following cell runs the Resume RAG Agent in async mode. Type your interview questions and get responses. If the answer is not found in your resume, you'll receive an SMS notification!

In [2]:
import asyncio

async def async_chat_loop(executor):
    print("Resume RAG Chatbot (Async Mode). Type your interview question (or 'exit' to quit):")
    while True:
        question = input("You: ")
        if question.lower() == "exit":
            print("Goodbye!")
            break
        print(f"[DEBUG] Agent received question: {question}")
        response = await asyncio.to_thread(executor.invoke, {"input": question})
        print("Bot:", response.get("output", response))

# To run the chat loop, uncomment the line below:
await async_chat_loop(executor)

Resume RAG Chatbot (Async Mode). Type your interview question (or 'exit' to quit):
[DEBUG] Agent received question: what are your skills?
[DEBUG] Resume search result: {'query': 'skills', 'result': 'The individual has skills in:\n\n1. Creating and managing secure practices for pipelines.\n2. Migrating teams from on-premise to cloud on Kubernetes.\n3. Big Data Architecture and Management.\n4. AWS (Amazon Web Services).\n5. Apache Spark.\n6. Kubernetes.\n7. Airflow.\n8. Java, Spring, Web Server, Hibernate.\n9. Understanding requirements and advising on technologies.'}
Bot: Based on my resume, my skills include:

1. Creating and managing secure practices for pipelines.
2. Migrating teams from on-premise to cloud on Kubernetes.
3. Big Data Architecture and Management.
4. AWS (Amazon Web Services).
5. Apache Spark.
6. Kubernetes.
7. Airflow.
8. Java, Spring, Web Server, Hibernate.
9. Understanding requirements and advising on technologies.
[DEBUG] Agent received question: Do you have a pate